# **ETLPipeline**

## Objectives

* Load healthcare insurance dataset from Kaggle (CSV format).
* Perform data cleaning: check nulls, handle duplicates, encode categorical features, and create BMI categories.
* Save the cleaned dataset for use in analysis and modeling notebooks.

## Inputs

* `insurance.csv` — the raw dataset downloaded from Kaggle, located in the `data/` folder.

## Outputs

* `cleaned_insurance.csv` — a cleaned and feature-engineered version of the dataset, saved in the `data/` folder.

## Additional Comments

* The BMI category feature is classified using WHO standard ranges: Underweight, Normal, Overweight, Obese.




### Data Cleaning

In [2]:
import numpy as np
import pandas as pd

In [10]:
# Load the dataset and cheking the first few rows
df = pd.read_csv('data/insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
# check the data types of the columns
print("\nData types:\n", df.dtypes)


Data types:
 age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object


In [ ]:
# Check for missing values
print("Missing values:\n", df.isnull().sum())

Missing values:
 age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [ ]:
# check for duplicates and print the count
duplicates = df.duplicated().sum()
print("Duplicate rows:", duplicates)

Duplicate rows: 1


In [16]:
# droping duplicate row
if duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicates dropped. New shape:", df.shape)

Duplicates dropped. New shape: (1337, 7)


In [17]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### Encode categorical variables

In [21]:
# Encoding categorical variables
df_encoded = pd.get_dummies(df, columns=['sex', 'smoker', 'region'])
df_encoded


,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,True,False,False,True,False,False,False,True
1,18,33.770,1,1725.55230,False,True,True,False,False,False,True,False
2,28,33.000,3,4449.46200,False,True,True,False,False,False,True,False
3,33,22.705,0,21984.47061,False,True,True,False,False,True,False,False
4,32,28.880,0,3866.85520,False,True,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,False,True,True,False,False,True,False,False
1334,18,31.920,0,2205.98080,True,False,True,False,True,False,False,False
1335,18,36.850,0,1629.83350,True,False,True,False,False,False,True,False
1336,21,25.800,0,2007.94500,True,False,True,False,False,False,False,True


In [24]:
# creating BMI categories
def categorize_bmi(bmi):
    if bmi < 18.5:
        return 'Under weight'
    elif 18.5 <= bmi < 24.9:
        return 'Normal weight'
    elif 25 <= bmi < 29.9:
        return 'Over weight'
    else:
        return 'Obesity'
    
df['bmi_category'] = df['bmi'].apply(categorize_bmi)
df

,age,sex,bmi,children,smoker,region,charges,bmi_category
0,19,female,27.900,0,yes,southwest,16884.92400,Over weight
1,18,male,33.770,1,no,southeast,1725.55230,Obesity
2,28,male,33.000,3,no,southeast,4449.46200,Obesity
3,33,male,22.705,0,no,northwest,21984.47061,Normal weight
4,32,male,28.880,0,no,northwest,3866.85520,Over weight
...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830,Obesity
1334,18,female,31.920,0,no,northeast,2205.98080,Obesity
1335,18,female,36.850,0,no,southeast,1629.83350,Obesity
1336,21,female,25.800,0,no,southwest,2007.94500,Over weight


In [ ]:
# Displaying the count of each BMI category
print(""df['bmi_category'].value_counts())

bmi_category
Obesity          721
Over weight      374
Normal weight    222
Under weight      20
Name: count, dtype: int64

### Saving cleaned data set to new csv file

In [26]:
cleaned_data = 'data/cleaned_insurance.csv'
df.to_csv(cleaned_data, index=False)
print(f'Cleaned data saved to {cleaned_data}')

Cleaned data saved to data/cleaned_insurance.csv
